## Vehicle counting 

Here, we follow steps to count the number of cars in the video


## Installing Ultralytics YOLO

##### install and import dependencies

In [1]:
!pip3 install torch torchvision torchaudio


Defaulting to user installation because normal site-packages is not writeable


In [2]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 14906, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 14906 (delta 11), reused 12 (delta 2), pack-reused 14880
Receiving objects: 100% (14906/14906), 13.93 MiB | 12.19 MiB/s, done.
Resolving deltas: 100% (10249/10249), done.


In [ ]:
ls

sample_data/  yolov5/


In [3]:
cd yolov5

/home/mojeee/Desktop/vehicle_counting/src/yolov5


In [4]:
ls

benchmarks.py    data/       LICENSE          requirements.txt  tutorial.ipynb
CITATION.cff     detect.py   models/          segment/          utils/
classify/        export.py   README.md        setup.cfg         val.py
CONTRIBUTING.md  hubconf.py  README.zh-CN.md  train.py


In [5]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 kB 6.1 MB/s eta 0:00:00
  Using cached matplotlib-3.6.2-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (9.4 MB)
  Using cached numpy-1.24.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached Pillow-9.3.0-cp38-cp38-manylinux_2_28_x86_64.whl (3.3 MB)
  Using cached psutil-5.9.4-cp36-abi3-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (280 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 17.1 MB/s eta 0:00:00
  Using cached scipy-1.9.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (33.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 44.9 MB/s eta 0:00:0031m54.1 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 52.1 MB/s eta 0:00:0

  Using cached absl_py-1.3.0-py3-none-any.whl (124 kB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.4/499.4 kB 43.8 MB/s eta 0:00:00
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached importlib_metadata-5.2.0-py3-none-any.whl (21 kB)
  Using cached MarkupSafe-2.1.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1


In [92]:
import torch
#from matplotlib import pyplot as plt
import numpy as np
import cv2

#### Load Model/


In [93]:
model = torch.hub.load("ultralytics/yolov5","yolov5s")

Using cache found in /home/mojeee/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-12-29 Python-3.8.10 torch-1.13.1+cu117 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


Lets take a look to the model

In [94]:
model

AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

## Making Detection 

at first lets load the my personal google drive to load the video

In [97]:
cap = cv2.VideoCapture("../data/HighWay4.mp4")

# skip some frames
cy1= 480;cy2=480;offset = 12;skframe = 0;leftcounter=0;rightcounter=0
while cap.isOpened():
    ret, frame = cap.read()
    
    # to skip some frames and make it faster  
    skframe +=1
    if skframe%3 !=0:
        continue
    frame = cv2.resize(frame, (1280,720))
    l1=140;l2=600;r1=670;r2=1090
    cv2.line(frame,(l1,cy1),(l2,cy1),(0,255,0),3)
    cv2.line(frame,(r1,cy2),(r2,cy2),(0,255,0),3)
    # detection
    results = model(frame)
    
    # find the center point of the box for counting issue
    locations = results.pandas().xyxy[0]
    for index,row in locations.iterrows():
        x1 = int(row['xmin'])
        y1 = int(row['ymin'])
        x2 = int(row['xmax'])
        y2 = int(row['ymax'])
        rectx1,recty1 = ((x1+x2)/2,(y1+y2)/2)
        rectcenter = int(rectx1),int(recty1)
        cx = rectcenter[0]
        cy = rectcenter[1]
        cv2.circle(frame,(cx,cy),3,(255,0,0),-1)
        if cy<(cy1+offset) and cy>(cy1-offset):
            if cx>l1 and cx<l2:
                leftcounter+=1
                cv2.line(frame,(l1,cy1),(l2,cy1),(0,0,255),3)
        if cy<(cy2+offset) and cy>(cy2-offset):
            if cx>r1 and cx<r2:
                rightcounter+=1
                cv2.line(frame,(r1,cy2),(r2,cy2),(0,0,255),3)
        x,y,w,h = 0,0,175,75
        cv2.rectangle(frame, (x, x), (x + w, y + h), (0,0,0), -1)
        cv2.putText(frame, str(leftcounter), (x + int(w/10),y + int(h/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
        cv2.putText(frame, "Left Line", (x +40+ int(w/10),y + int(h/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
        x,y,w,h = 1063,0,175,75
        cv2.rectangle(frame, (x, 0), (x + w, y + h), (0,0,0), -1)
        cv2.putText(frame, str(rightcounter), (x + int(w/10),y + int(h/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
        cv2.putText(frame, "Right Line", (x +40+ int(w/10),y + int(h/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
        #cv2.putText(frame, str(leftcounter), (150, 150),cv2.FONT_HERSHEY_SIMPLEX,1,(255, 0, 0),1,2 )
    
    cv2.imshow("Yolo Results",np.squeeze(results.render()))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



# Tracking Objects

KeyError: 0

## Counting the number of cars